In [45]:
import os
import json
import pandas as pd
import json
from tqdm import tqdm
from config import EOS, PAD
from dataset import get_dataset
from sklearn.metrics import accuracy_score

### Analyze financial data

In [24]:
FIN_DATA = './data/financial_data'
paths = [
    os.path.join(FIN_DATA, 'Sentences_AllAgree.txt'), 
    os.path.join(FIN_DATA, 'Sentences_75Agree.txt'), 
    os.path.join(FIN_DATA, 'Sentences_66Agree.txt'), 
    os.path.join(FIN_DATA, 'Sentences_50Agree.txt')
]

sentences = set()
labels = set()
words = set()
data = []

for p in paths:
    with open(p, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            if line not in sentences:
                sentence = line.strip().lower()
                if '.@' in sentence:
                    prompt = sentence.split('.@')[0]
                    label = sentence.split('.@')[-1]
                else:
                    prompt = sentence.split('@')[0]
                    label = sentence.split('@')[-1]

                data.append([prompt, label])
                labels.add(label)
                for word in prompt.split(' '):
                    if '@' in prompt:
                        print(prompt)
                    words.add(word)
            sentences.add(line)

df = pd.DataFrame(data=data, columns=['sentence', 'label'], index=None)
# df.to_csv('./data/financial_data.csv')

print(words)
print()
print(labels)

VOCAB_SIZE = len(words) + len(labels) + 2
print(VOCAB_SIZE)

{'', 'abidjan', 'marski', 'mformation', 'java', 'eero', 'lock', 'launches', 'debt', 'consumers', 'function', 'nadu', 'circulation', '4,442.10', 'cl51', '60-percent', 'strand', 'aura', 'mills', 'rise', '0.4', '2,518.67', 'html', 'spanning', 'rebuilt', 'showed', 'billingham', 'alloys', 'necessary', 'handlers', 'airtel', 'region', 'relations', 'altimo', 'eur3', 'takanen', 'position', 'eye', 'ingredient', 'leed', 'camera', 'loss', 'applying', 'fortune', 'standard', 'chl', 'las', 'operates', 'stackers', 'scissor', 'bad', 'aligned', 'vnh', 'erection', 'opened', 'nsn', 'governemtn', 'fulfillment', 'fairly', 'fat', 'kito', 'reopen', '23,335', '8.5', 'of2008', 'kraski', 'refurbishing', 'hard', 'redundancy', 'vaccines', 'ft', 'process-related', 'generate', 'liability', 'year-on-year', 'jorgen', 'bollore', 'journey', 'pacts', '37.1', 'eng', 'postimees', 'hevea', 'casualty', 'battery', 'gross', '6,800', 'catalysts', 'dominating', 'vuorenmaa', 'investment', 'scheme', 'lordi-cola', '44', 'newly-form

In [26]:
sne_size = set()

for sentence in sentences:
    # print(sentence)
    sne_size.add(len(sentence.split(' ')))

print(sne_size)
print(max(sne_size))

max_len = max(sne_size)

{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 60, 62, 81}
81


In [27]:
padded_sentences = []
rows = []

for i, row in df.iterrows():
    sentence = row['sentence']
    label = row['label']

    if len(sentence.split(' ')) != max_len:
        for j in range(len(sentence.split(' ')), max_len - 1):
            sentence += f' {PAD}'
        sentence += f' {EOS}'

    rows.append([sentence, label])

df_filtered = pd.DataFrame(data=rows, columns=['sentence', 'label'], index=None)

In [29]:
sentence_sizes = set()

for i, row in df_filtered.iterrows():
    sentence = row['sentence']
    if len(sentence.split(' ')) < max_len or len(sentence.split(' ')) > max_len:
        print(len(sentence.split(' ')))
    sentence_sizes.add(len(sentence.split(' ')))

In [30]:
sentence_sizes

{81}

In [31]:
df_filtered.to_csv(os.path.join('data', 'financial_data.csv'))

In [32]:
words = set()

for i, row in df_filtered.iterrows():
    sentence = row['sentence']
    for word in sentence.split(' '):
        words.add(word)

print(words)

{'', 'abidjan', 'marski', 'mformation', 'java', 'eero', 'lock', 'launches', 'debt', 'consumers', 'function', 'nadu', 'circulation', '4,442.10', 'cl51', '60-percent', 'strand', 'aura', 'mills', 'rise', '0.4', '2,518.67', 'html', 'spanning', 'rebuilt', 'showed', 'billingham', 'alloys', 'necessary', 'handlers', 'airtel', 'region', 'relations', 'altimo', 'eur3', 'takanen', 'position', 'eye', 'ingredient', 'leed', 'camera', 'loss', 'applying', 'fortune', 'standard', 'chl', 'las', 'operates', 'stackers', 'scissor', 'bad', 'aligned', 'vnh', 'erection', 'opened', 'nsn', 'governemtn', 'fulfillment', 'fairly', 'fat', 'kito', 'reopen', '23,335', '8.5', 'of2008', 'kraski', 'refurbishing', 'hard', 'redundancy', 'vaccines', 'ft', 'process-related', 'generate', 'liability', 'year-on-year', 'jorgen', 'bollore', 'journey', 'pacts', '37.1', 'eng', 'postimees', 'hevea', 'casualty', 'battery', 'gross', '6,800', 'catalysts', 'dominating', 'vuorenmaa', 'investment', 'scheme', 'lordi-cola', '44', 'newly-form

In [35]:
len(words)

11736

In [41]:
word_tokens = {}

for i, word in enumerate(words):
    word_tokens[word] = int(i)

with open(os.path.join('data', 'word_tokens.json'), 'w') as f:
    json.dump(word_tokens, f, indent=4)

In [42]:
features, labels = get_dataset()

In [44]:
features_len = set()

for f in features:
    features_len.add(len(f))

features_len

{81}

In [48]:
test_df = pd.read_csv('inf.csv')
labels = []
preds = []

for i, row in test_df.iterrows():
    labels.append(row['ground_truth'])
    preds.append(row['prediction'])

# Calculate accuracy
accuracy = accuracy_score(labels, preds)

# Print the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 80.99%
